# Семинар 12 - Ранжирование

In [ ]:
import numpy as np
import pandas as pd

seed = 42
np.random.seed(seed)

# Pointwise - TF-IDF

## The Movies Dataset
[Исходники данных](https://www.kaggle.com/rounakbanik/the-movies-dataset) для работы на семинаре.

In [ ]:
# загрузим данные о фильмах (набор документов)

metadata = pd.read_csv('data/movies/movies_metadata.csv', low_memory=False)
metadata.head(2)

In [ ]:
# загрузим данные о фильмах (оценки релевантности)
rating = pd.read_csv('data/movies/ratings_small.csv', low_memory=False)

rating.head(2)

### Неперсонализированная рекомендация

Сделаем свой аналог [IMDb rating](https://www.imdb.com/chart/top?ref_=nb_mv_3_chttp) через [взвешенный рейтинг](https://www.quora.com/How-does-IMDbs-rating-system-work):
$$WeightedRating=(\frac{v}{v+m}⋅R)+(\frac{m}{v+m}⋅C)$$

где:
- v (votes) число оценок фильма;
- m (minimum) минимальное число оценок для попадания в топ;
- R (rating) средний рейтинг фильма;
- C (across) средний рейтинг по всем фильмам.

In [ ]:
C = metadata['vote_average'].mean()
print(C)

m = metadata['vote_count'].quantile(0.90)
print(m)

q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
print(q_movies.shape)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
# Фильмы, основанные на баллах, рассчитанных выше
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

Получилось достаточно близко к оригинальному топу.


О чем это говорит?

## Content-based рекомендация

Попробуем сделаем рекомендательную систему на основе описания фильмов.

In [ ]:
metadata['overview'].head()

In [ ]:
first_n =  metadata.copy()[:30000]

### Найдем векторное представление данных - TF-IDF

Рассмотрим частотное представление слов через TF-IDF.

TF-IDF (сокращение от term frequency — inverse document frequency) – это статистическая мера для оценки важности слова в документе, который является частью коллекции или корпуса.

Скоринг по TF-IDF растет пропорционально частоте появления слова в документе, но это компенсируется количеством документов, содержащих это слово.

Формула скоринга для слова X в документе Y:
![](http://3.bp.blogspot.com/-u928a3xbrsw/UukmRVX_JzI/AAAAAAAAAKE/wIhuNmdQb7E/s1600/td-idf-graphic.png)

TF (term frequency — частота слова) – отношение числа вхождений слова к общему числу слов документа.

![](https://habrastorage.org/r/w1560/webt/ai/p0/wk/aip0wkqcynj8q1cxwxlufspqqds.png)

IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции.

![](https://habrastorage.org/r/w1560/webt/6j/xd/32/6jxd32ydlpkmixkjw6hdgmp6f6m.png)

В итоге, вычислить TF-IDF для слова term можно так:

![](https://habrastorage.org/r/w1560/webt/hl/tp/n0/hltpn0vg_gdo8bn1pfimbvu60no.png)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# ограничим размер словаря до 5000 элементов
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

In [ ]:
first_n['overview'] = first_n['overview'].fillna('')

In [ ]:
tfidf_matrix = tfidf.fit_transform(first_n['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
tfidf.get_feature_names()[500:510]

### Оценим схожесть полученных векторов

Схожесть будем измерять по косинусной метрике

In [ ]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T).toarray()

In [ ]:
cosine_sim.shape

In [ ]:
print(cosine_sim[0])

In [ ]:
indices = pd.Series(firast_n.index, index=firast_n['title']).drop_duplicates()

In [ ]:
indices

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Получить индекс фильма, соответствующий названию
    idx = indices[title]

    # Взять парные оценки сходства всех фильмов с этим фильмом
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Сортировать фильмы по сходству
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Взять оценки 10 самых похожих фильмов
    sim_scores = sim_scores[1:11]

    # Получить индексы фильма
    movie_indices = [i[0] for i in sim_scores]

    # Вернуть 10 самых похожих фильмов
    return metadata['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

### Сделеаем рекомендацию для пользователя

In [ ]:
rating = rating.drop(['timestamp'], axis=1)
rating

In [ ]:
# возьмем одного пользователя
user_id = 1

user_rating = rating[rating['userId'] == user_id].drop(['userId'], axis=1).sort_values(by=['rating'], ascending=False)
user_rating

In [ ]:
user_rating["movieId"].values[0]

In [ ]:
firast_n["title"].values[1172]

In [ ]:
indices["Army of Darkness"]

In [ ]:
get_recommendations("Army of Darkness")

In [ ]:
firast_n["title"].values[user_rating["movieId"].values]

# Pairwise - RankNet

Функция ошибки по паре объектов (в пару к запросу).

$\displaystyle \sum_q \sum_{i, j:\ r^q_i \gt r^q_j} l(f({x}^q_i) - f({x}^q_j)) \to \min$

В качестве функции для оптимизации мы берём классическую функцию потерь: кросс-энтропию $C$:
$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$

$o_i$ — предсказание нашего алгоритма для одного объекта (*логит* или *скор*):

$o_i \equiv f(x_i)$,

$o_{ij}=f(x_i)-f(x_j)$

Для превращения этого в вероятность, т.е. нормирования в интервал $[0, 1]$, мы можем воспользоваться обычной логистической функцией. Разность логитов будем использовать как степень для числа $e$:

$\displaystyle P_{ij} \equiv \frac {e^{o_{ij}}} {1 + e^{o_{ij}}}$ — функция отображения предсказания (логита) в вероятность.


Тогда функцию потерь, или функцию стоимости (cost function) можно переписать следующим образом:

$C_{ij} = -\overline P_{ij} o_{ij} + \log(1 + e^{o_{ij}})$

In [ ]:
import torch

In [ ]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [ ]:
ranknet_model = RankNet(num_input_features=10)

In [ ]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_2

In [ ]:
preds = ranknet_model(inp_1, inp_2)
preds

In [ ]:
first_linear_layer = ranknet_model.model[0]

In [ ]:
first_linear_layer.weight.grad

In [ ]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [ ]:
first_linear_layer.weight.grad

In [ ]:
first_linear_layer = ranknet_model.model[0]

In [ ]:
first_linear_layer.weight.grad

In [ ]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [ ]:
first_linear_layer.weight.grad

In [ ]:
ranknet_model.zero_grad()

# Listwise - ListNet

В listwise, как следует из названия, мы должны использовать функцию потерь, которая рассчитывается на всём множестве релевантных запросу документов.

Можем говорить, что представлено полное множество перестановок $\Omega_n$, указывая размер множества объектов, на которых рассчитываются перестановки $n$.

Каждая перестановка $\pi$  характеризуется полным указанием, какой объект стоит на первой, на второй и так далее до позиции $n$.

$\pi = \langle \pi(1), \pi(2), ..., \pi(n) \rangle$

Каждое $\pi_i$ указывает на конкретный объект в перестановке.

$\displaystyle P_s (\pi) = \prod^n_{j = 1} \frac {\phi(s_{\pi(j)})} {\sum^n_{k = j} \phi(s_{\pi(k)})}$ — вероятность возникновения такой перестановки

И в числителе, и в знаменателе к скору, или к логиту, $j$-го объекта конкретной перестановки $\pi_i$ применяется функция преобразования скоров.

К этой функции указываются следующие требования:

- Возрастающая;
- Строго положительная.

То есть, чем больше логит, тем выше значение этой функции, при этом ни при каких обстоятельствах она не может стать отрицательной (*иначе бы мы могли получать отрицательные вероятности, чего быть не может*).

Под эти требования подходит много функций, но самая распространенная — экспонента, то есть возведение e в степень логита с индексом $\pi_j$.

Рассмотрим знаменатель: здесь сумма от $j$-го до $n$-го (последнего) объекта, суммируем мы в точности те же значения, что и в числителе — некоего рода нормализация.

Смотрим, какую долю от суммы всех скоров составляет наш текущий $j$-й объект.

Выводы для метода:

- Наибольшая вероятность у перестановки, в которой объекты отсортированы в порядке убывания.
- Наименьшая вероятность у перестановки, в которой объекты отсортированы в порядке возрастания.
- Количество перестановок равно $n!$ (много).

Благодаря SoftMax не нужно считать все перестановки — можно получить скоры и преобразовать их в TopOneProbability.

Для обучения нашего алгоритма, для расчёта градиентов и градиентного спуска по весам, можно использовать любую функцию потерь, которая оптимизирует расстояние между двумя распределениями вероятностей.

Это может быть классическая кросс-энтропия:

$\displaystyle L(y^{(i)}, z^{(i)}) = -\sum^n_{j = 1} P_{y^{(i)}}(j) \log(P_{z^{(i)}}(j))$,

In [ ]:
from itertools import combinations

from utils import ndcg, num_swapped_pairs

In [ ]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )


    def forward(self, input_1):
        logits = self.model(input_1)
        return logits


In [ ]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """

    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))


def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)

    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)

    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)

    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)

#     bins = [-1, 1]  # 3 relevances
    bins = [-1, 0, 1, 2]  # 5 relevances
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )

    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [ ]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())

In [ ]:
torch.unique(ys_train)

In [ ]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)

    X_train = X_train[idx]
    ys_train = ys_train[idx]

    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size

        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            batch_loss = listnet_kl_loss(batch_ys, batch_pred)
#             batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()

        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: " 
                  f"{valid_swapped_pairs}/{N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}")